In [167]:
import pandas as pd
initial_dataset = pd.read_csv("direct_marketing.csv")
initial_dataset.sample(10)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,DM_category
1679,10,1) $0 - $100,29.99,1,0,Urban,0,Web,Mens E-Mail,0,0,0.00,2
44204,11,1) $0 - $100,65.67,1,0,Rural,1,Web,Mens E-Mail,1,1,147.19,2
49984,7,1) $0 - $100,73.94,0,1,Urban,0,Phone,No E-Mail,0,0,0.00,5
26553,3,5) $500 - $750,599.73,0,1,Urban,1,Web,No E-Mail,0,0,0.00,5
45350,3,1) $0 - $100,33.81,0,1,Surburban,0,Web,Womens E-Mail,0,0,0.00,1
59042,2,1) $0 - $100,29.99,1,0,Surburban,1,Web,Mens E-Mail,0,0,0.00,2
62046,6,1) $0 - $100,53.79,0,1,Rural,1,Phone,Womens E-Mail,0,0,0.00,1
23930,8,1) $0 - $100,34.07,1,0,Urban,1,Phone,No E-Mail,0,0,0.00,6
21065,6,4) $350 - $500,392.43,0,1,Urban,0,Web,Mens E-Mail,0,0,0.00,3
54623,5,1) $0 - $100,29.99,1,0,Urban,0,Phone,Womens E-Mail,0,0,0.00,4


In [169]:
initial_dataset.shape

(64000, 13)

The table above shows the original dataset we used. This dataset contains 64000 customers who purchased within a year. Some of them received e-mail campain featuring Womens or Mens mechandise but some of them didn't receive any email. These customers were tracked in order to see how the advertising e-mails affected their behavior.

<img src="./starSchemaExample.jpg">

The diagram above represents a star schema including just a few columns from the original dataset for the sake of simplicity.

In [3]:
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
import dautil as dl
from tabulate import tabulate
import sqlite3
import os
from joblib import Memory

Base = declarative_base()
memory = Memory(location='.')

In [4]:
#Define a class for the Zip code dimension
class DimZipCode(Base):
    __tablename__ = 'dim_zip_code'
    id = Column(Integer, primary_key=True)
    # Urban, Suburban, or Rural.
    zip_code = Column(String(8), nullable=False, unique=True)

In [5]:
#Define a class for the segment dimension
class DimSegment(Base):
    __tablename__ = 'dim_segment'
    id = Column(Integer, primary_key=True)
    # Mens E-Mail, Womens E-Mail or No E-Mail
    segment = Column(String(14), nullable=False, unique=True)

In [6]:
#Define a class for the channel dimension
class DimChannel(Base):
    __tablename__ = 'dim_channel'
    id = Column(Integer, primary_key=True)
    channel = Column(String)

In [7]:
#Define a class for the fact table
class FactSales(Base):
    __tablename__ = 'fact_sales'
    id = Column(Integer, primary_key=True)
    zip_code_id = Column(Integer, ForeignKey('dim_zip_code.id'),
                         primary_key=True)
    segment_id = Column(Integer, ForeignKey('dim_segment.id'),
                        primary_key=True)
    channel_id = Column(Integer, ForeignKey('dim_channel.id'),
                        primary_key=True)

    # Storing amount as cents
    spend = Column(Integer)

    def __repr__(self):
        return "zip_code_id={0} channel_id={1} segment_id={2}".format(
            self.zip_code_id, self.channel_id, self.segment_id)


In [8]:
#Define the following function to create a SQLAlchemy session:
def create_session(dbname):
    engine = create_engine('sqlite:///{}'.format(dbname))
    DBSession = sessionmaker(bind=engine)
    Base.metadata.create_all(engine)

    return DBSession()

In [9]:
#Define the following function to populate the segment dimension table:
def populate_dim_segment(session):
    options = ['Mens E-Mail', 'Womens E-Mail', 'No E-Mail']

    for option in options:
        if not dl.db.count_where(session, DimSegment.segment, option):
            session.add(DimSegment(segment=option))

    session.commit()

In [10]:
#Define the following function to populate the ZIP code dimension table:
def populate_dim_zip_code(session):
    # Note the interesting spelling
    options = ['Urban', 'Surburban', 'Rural']

    for option in options:
        if not dl.db.count_where(session, DimZipCode.zip_code, option):
            session.add(DimZipCode(zip_code=option))

    session.commit()

In [11]:
#Define the following function to populate the channel dimension table:
def populate_dim_channels(session):
    options = ['Phone', 'Web', 'Multichannel']

    for option in options:
        if not dl.db.count_where(session, DimChannel.channel, option):
            session.add(DimChannel(channel=option))

    session.commit()

In [12]:
#Define the following function to populate the fact table (using straight SQL):
def load(csv_rows, session, dbname):
    channels = dl.db.map_to_id(session, DimChannel.channel)
    segments = dl.db.map_to_id(session, DimSegment.segment)
    zip_codes = dl.db.map_to_id(session, DimZipCode.zip_code)
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    logger = dl.log_api.conf_logger(__name__)

    for i, row in enumerate(csv_rows):
        channel_id = channels[row['channel']]
        segment_id = segments[row['segment']]
        zip_code_id = zip_codes[row['zip_code']]
        spend = dl.data.centify(row['spend'])

        insert = "INSERT INTO fact_sales (id, segment_id,\
            zip_code_id, channel_id, spend) VALUES({id}, \
            {sid}, {zid}, {cid}, {spend})"
        c.execute(insert.format(id=i, sid=segment_id,
                                zid=zip_code_id, cid=channel_id,     spend=spend))


        if i % 1000 == 0:
            logger.info("Progress %s/64000", i)
            conn.commit()

    conn.commit()
    c.close()
    conn.close()

In [13]:
#Define the following function to download and parse the data:
@memory.cache
def get_and_parse():
    out = dl.data.get_direct_marketing_csv()
    return dl.data.read_csv(out)

In [171]:
#The following block uses the functions and classes we defined:
if __name__ == "__main__":
  
    dbname = './marketing.db'
    session = create_session(dbname)
    populate_dim_segment(session)
    populate_dim_zip_code(session)
    populate_dim_channels(session)

    if session.query(FactSales).count() < 64000:
        load(get_and_parse(), session, dbname)
   
    
    
    fsum = func.sum(FactSales.spend)
    
    query = session.query(DimSegment.segment, DimChannel.channel,
                          DimZipCode.zip_code, fsum)
    
    dim_cols = (DimSegment.segment, DimChannel.channel, DimZipCode.zip_code)
    dim_entities = [dl.db.entity_from_column(col) for col in dim_cols]
    
    #Put all tables together
    spend_totals = query.join(DimSegment, FactSales.segment_id==DimSegment.id)\
                        .join(DimChannel, FactSales.channel_id==DimChannel.id)\
                        .join(DimZipCode, FactSales.zip_code_id==DimZipCode.id)\
                        .group_by(*dim_cols).order_by(fsum.desc()).all()
    
    print(tabulate(spend_totals, tablefmt='psql',
                   headers=['Segment', 'Channel', 'Zip Code', 'Spend']))

+---------------+--------------+------------+---------+
| Segment       | Channel      | Zip Code   |   Spend |
|---------------+--------------+------------+---------|
| Mens E-Mail   | Web          | Surburban  |  753965 |
| Mens E-Mail   | Phone        | Urban      |  475161 |
| Womens E-Mail | Web          | Urban      |  465327 |
| Mens E-Mail   | Phone        | Surburban  |  447702 |
| Mens E-Mail   | Web          | Urban      |  437970 |
| Womens E-Mail | Phone        | Urban      |  397354 |
| Womens E-Mail | Web          | Surburban  |  333875 |
| No E-Mail     | Phone        | Surburban  |  292136 |
| No E-Mail     | Web          | Surburban  |  289213 |
| Womens E-Mail | Phone        | Surburban  |  273985 |
| Mens E-Mail   | Web          | Rural      |  252286 |
| No E-Mail     | Web          | Urban      |  244141 |
| Mens E-Mail   | Multichannel | Urban      |  224474 |
| Womens E-Mail | Web          | Rural      |  218235 |
| Womens E-Mail | Multichannel | Urban      |  2